In [1]:
import xgboost as xgb
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')
#from feature_engineering import add_retning
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import lightgbm as lgb
import sklearn as skl
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
sys.path.append('../data')

from display_data import import_data

HEr kommer funksjonene

In [57]:
def add_parks_and_schools(df):
    """with parks we are going to add all as radius to different parks, will also do this with schools"""
    parks = pd.read_csv('parks_cordinates.csv')
    schools = pd.read_csv('school_cordinates.csv')
    parks = convert_from_coordinates_to_lat_long(parks)
    schools = convert_from_coordinates_to_lat_long(schools)
    
    total = remove_duplicates(parks,schools)

    for i, cordinate in enumerate(total):
        df['radius'+ str(i)] = np.sqrt((cordinate[0]-df['latitude'])**2 + (cordinate[1]-df['longitude'])**2)
        df = add_direction(df, latitude = cordinate[0], longitude = cordinate[1], name = str(i))


    return df



def remove_duplicates(parks,schools):
    """we want to remove places which is really close"""
    schools = schools.drop(schools[schools['school']==5].index)
    schools = schools.drop(schools[schools['school']==7].index)
    schools = schools.drop(schools[schools['school']==3].index)
    parks = parks.drop(parks[parks['park']==4].index)
    parks = parks.drop(parks[parks['park']==14].index)

    return np.concatenate((schools[['latitude','longitude']].to_numpy(),parks[['latitude','longitude']].to_numpy()))

def closest_park(df):
    parks = pd.read_csv('parks_cordinates.csv')
    parks = convert_from_coordinates_to_lat_long(parks)

    df = find_closest(df,parks,'park')

    return df

def closest_school(df):
    schools = pd.read_csv('school_cordinates.csv')
    schools = convert_from_coordinates_to_lat_long(schools)

    df = find_closest(df,schools,'school')

    return df

def closest_attraction(df):
    attractions = pd.read_csv('attractions_cordinates.csv')
    attractions = convert_from_coordinates_to_lat_long(attractions)

    df = find_closest(df,attractions,'atraction')

    return df

    return df

def closest_school(df):
    schools = pd.read_csv('school_cordinates.csv')
    schools = convert_from_coordinates_to_lat_long(schools)

    df = find_closest(df,schools,'school')

    return df

def closest_attraction(df):
    attractions = pd.read_csv('attractions_cordinates.csv')
    attractions = convert_from_coordinates_to_lat_long(attractions)

    df = find_closest(df,attractions,'atraction')

    return df

def closest_attraction(df):
    attractions = pd.read_csv('attractions_cordinates.csv')
    attractions = convert_from_coordinates_to_lat_long(attractions)

    df = find_closest(df,attractions,'atraction')

    return df

def closest_train(df):
    trains = pd.read_csv('train_cordinates.csv')
    trains = convert_from_coordinates_to_lat_long(trains)

    df = find_closest(df,trains,'train_station')

    return df

def closest_metro(df):
    metroes = pd.read_csv('metro_cordinates.csv')
    metroes = convert_from_coordinates_to_lat_long(metroes,metro=True)

    df = find_closest(df, metroes,'metro_station')
    return df

def find_closest(df,df_with_coordinates,text):
    """finds closets location for each location in df to all the locations in df_with_coordinates"""
    df[['closest_' + text,'distance_' + text]] = df.apply(closest,args=(df_with_coordinates,text),axis=1)
    df['closest_' + text] = df['closest_' + text].astype(float)
    return df

def closest(df, list_of_coordinates_and_category,text):
    if df.id%1000==0:
        print(df.id)
    """finds the closets coordinate in list of coordinates and the given coordinate"""
    point= df[['latitude','longitude']].to_numpy()
    list_of_coordinates_and_category['distances'] = list_of_coordinates_and_category.apply(calculate_distance,args=(point[0],point[1]),axis=1)
    shortest_row = list_of_coordinates_and_category['distances'].idxmin()
    return list_of_coordinates_and_category.iloc[shortest_row][[text,'distances']]
    #calculate_distance(coordinate, list_of_coordinates_and_category[['longitude','latitude']])

def calculate_distance(df,pointlat,pointlong):
    """calculates the distance between two points"""
    distance = np.sqrt((pointlat-df['latitude'])**2 + (pointlong-df['longitude'])**2)
    return distance

def convert_from_coordinates_to_lat_long(df,metro=False):
    if metro:
        df[['latitude','longitude']] = df['coordinates'].str.split(" ",1, expand=True)
        df[['latitude','longitude']] = df[['latitude','longitude']].astype(float)
    else:
        df[['latitude','longitude']] = df['coordinates'].str.split(',',1, expand=True)
        df[['latitude','longitude']] = df[['latitude','longitude']].astype(float)

    return df

def fix_missing_value(df):
    #layout, set everythin to 1
    df['layout'].fillna(int(1.0),inplace=True)



    #found correleation between number of stories and balconies,loggias
    df.loc[(df['stories'] < 7.0)&df['loggias'].isna(),'loggias'] = 0
    df.loc[(df['stories'] < 7.0)&df['balconies'].isna(),'balconies'] = 1

    df.loc[((df['stories'] > 12) & (df['stories'] < 16))|((df['stories'] > 17) & (df['stories'] < 24))&(df['loggias'].isna()),'loggias'] = 1
    df.loc[((df['stories'] > 12) & (df['stories'] < 16))|((df['stories'] > 17) & (df['stories'] < 24))&(df['loggias'].isna()),'balconies'] = 0


    #use the correleation between balconies and loggias to replace nans,
    df.loc[(df['loggias'] == 0.0 )& df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['balconies'] == 0.0 )& df['loggias'].isna(),'loggias'] = 1
    df.loc[(df['loggias'] > 0.0 )& df['balconies'].isna(),'balconies'] = 0
    df.loc[(df['balconies'] > 0.0 )& df['loggias'].isna(),'loggias'] = 0

    #use correlation between material and ceiling groups
    ceiling_under_2 = len(df[((df['material'] == 4)|(df['material'] == 3)) & df['ceiling'].isna()])
    sample = df[df['ceiling'] < 2.6]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[((df['material'] == 4)|(df['material'] == 3)) & (df['ceiling'].isna()),'ceiling'] = sample
    #use corelation between condition and ceiling groups
    ceiling_under_2 = len(df[(df['condition'] == 1) & df['ceiling'].isna()])
    sample = df[df['ceiling'] < 2.6]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[(df['condition'] == 1) & df['ceiling'].isna(),'ceiling'] = sample


    #use correleation betwwen ceiling and area
    ceiling_over_3 = len(df[(df['area_total'] > 104)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] > 3.0]['ceiling'].sample(n=ceiling_over_3,replace=True).to_numpy()
    df.loc[(df['area_total'] > 104)&(df['ceiling'].isna() ),'ceiling'] = sample

    ceiling_over_3 = len(df[(df['area_total'] > 75)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] > 2.95]['ceiling'].sample(n=ceiling_over_3,replace=True).to_numpy()
    df.loc[(df['area_total'] > 75)&(df['ceiling'].isna() ),'ceiling'] = sample

    ceiling_under_2 = len(df[(df['area_total'] < 55)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] < 2.8]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[(df['area_total'] < 55)&(df['ceiling'].isna() ),'ceiling'] = sample

    #seller
    #use correleation between seller and new
    df.loc[(df['new'] == 1 )& df['seller'].isna(),'seller'] = 3

    #use correleation between seller and constructed
    df.loc[(df['constructed'] > 2018)& df['seller'].isna(),'seller'] = 3
    df.loc[(df['constructed'] < 1990)& df['seller'].isna(),'seller'] = 1

    #use correleation between seller and stories
    df.loc[(df['stories'] > 17)& df['seller'].isna(),'seller'] = 3
    df.loc[(df['stories'] < 9)& df['seller'].isna(),'seller'] = 1

    #use correleation between seller and district
    df.loc[((df['district'] == 3)|(df['district']==6))& df['seller'].isna(),'seller'] = 1
    df.loc[((df['district'] == 11)|(df['district']==2))& df['seller'].isna(),'seller'] = 3

    #conditon
    #use correleation between condition and new
    df.loc[(df['new'] == 1)& df['condition'].isna(),'condition'] = 0

    #use correleation between condition and constructed year
    df.loc[(df['constructed'] >= 2017)& df['condition'].isna(),'condition'] = 0
    df.loc[(df['constructed'] <= 1989)& df['condition'].isna(),'condition'] = 1

    #material
    #use correlation between material and district
    df.loc[(df['district'] == 3)& df['material'].isna(),'material'] = 3

    #use correleation between material and constructed year
    df.loc[((df['constructed'] > 1970)& (df['constructed'] <= 1989))&df['material'].isna(),'material'] = 3
    df.loc[(df['constructed'] <= 1970)&df['material'].isna(),'material'] = 0

    #use correleation between stories and material
    df.loc[(df['stories'] < 7)& df['material'].isna(),'material'] = 0
    df.loc[((df['stories'] > 7)&(df['stories'] < 9))& df['material'].isna(),'material'] = 3

    #use correleation between conditon and material
    df.loc[(df['condition'] == 3)& df['material'].isna(),'material'] = 3
    df.loc[(df['condition'] == 0)& df['material'].isna(),'material'] = 2

    #use correleation between parking and material
    df.loc[((df['parking'] == 3)|(df['parking'] == 2))& df['material'].isna(),'material'] = 2

    #user correlation between area and material
    df.loc[(df['area_total'] > 104)& df['material'].isna(),'material'] = 2

    #parking
    #use coreletion between parking and material
    df.loc[((df['material'] == 0)|(df['material'] == 3))& df['parking'].isna(),'parking'] = 1

    #use correleation between stories and parking
    df.loc[((df['stories'] > 30))& df['parking'].isna(),'parking'] = 0
    df.loc[((df['stories'] > 7)&(df['stories'] < 9))& df['parking'].isna(),'parking'] = 1

    #use correleation between radius and parking
    df.loc[(df['radius'] < 0.0547)& df['parking'].isna(),'parking'] = 0
    df.loc[(df['radius'] > 0.251)& df['parking'].isna(),'parking'] = 1

    #windows_court and windows_street
    #windows_street and balconies
    df.loc[(df['balconies'] == 2)& df['windows_street'].isna(),'windows_street'] = 1

    #windows street and area
    df.loc[(df['area_total'] > 106)& df['windows_street'].isna(),'windows_street'] = 1
    df.loc[(df['area_total'] < 47)& df['windows_street'].isna(),'windows_street'] = 0

    #windwos_street and stories
    df.loc[(df['stories'] > 40)& df['windows_street'].isna(),'windows_street'] = 1
    df.loc[(df['stories'] < 9)& df['windows_street'].isna(),'windows_street'] = 0

    #windows_street and windows_court
    df.loc[(df['windows_street'] == 0)& df['windows_court'].isna(),'windows_court'] = 1

    df.loc[((df['windows_street'] == 1)& (df['area_total'] > 75)) & df['windows_court'].isna(),'windows_court'] = 1
    df.loc[((df['windows_street'] == 1)& (df['area_total'] < 47)) & df['windows_court'].isna(),'windows_court'] = 0

    #balconies
    #balconies and stories
    df.loc[((df['stories'] >= 12)& (df['area_total'] <= 16)) & df['balconies'].isna(),'balconies'] = 0
    df.loc[(df['stories'] < 7) & df['balconies'].isna(),'balconies'] = 1
    df.loc[((df['stories'] > 17)& (df['area_total'] <= 24)) & df['balconies'].isna(),'balconies'] = 0

    #matrial and balconies
    df.loc[(df['material'] == 0) & df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['material'] == 2) & df['balconies'].isna(),'balconies'] = 0

    #balconies and constructed
    df.loc[(df['constructed'] < 1970) & df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['constructed'] > 2018) & df['balconies'].isna(),'balconies'] = 0

    #loggias
    #loggias and balconies
    df.loc[(df['balconies'] == 0) & df['loggias'].isna(),'loggias'] = 1
    df.loc[(df['balconies'] > 0) & df['loggias'].isna(),'loggias'] = 0

    #loggias and stories
    df.loc[(df['stories'] < 7) & df['loggias'].isna(),'loggias'] = 0
    df.loc[((df['stories'] >= 17)& (df['area_total'] <= 24)) & df['loggias'].isna(),'loggias'] = 1

    #bathrooms shared, bathroomes private

    #loggias and blaconies



    return df

def add_radius(df):
    #adds radius column to dataframe
    df['radius'] = np.sqrt((df['latitude']-55.75)**2 + (df['longitude']-37.55)**2)
    return df

def group_by_feature_and_price(df, feature, data_train = pd.DataFrame()):
    #makes a new column with the mean price for each group to all rows
    if data_train.empty:
        grouped = df.groupby([feature])

        mean_price = grouped['price'].mean()

        df_merged = pd.merge(df,mean_price, on=feature, how='left')

        column_name = str(feature + '_price')

        df[column_name] = df_merged['price_y']
    else:
        grouped = data_train.groupby([feature])

        mean_price = grouped['price'].mean()

        df_merged = pd.merge(df,mean_price, on=feature, how='left')

        column_name = str(feature + '_price')

        df[column_name] = df_merged['price']
    return df

def one_hot_encode(df, column_to_encode):
    """one hots encode for one singel column"""
    encoded_df = pd.get_dummies(df[[column_to_encode]].astype(str))
    df = pd.concat([df,encoded_df],axis=1)
    return df

def one_hot_encode_multiple(df, list_of_columns):
    """takes in multiple columns and runs one hot encode for each column"""
    for column_to_encode in list_of_columns:
        #print(column_to_encode)
        df = one_hot_encode(df, column_to_encode)
    return df

def add_direction(df, latitude=55.75, longitude=37.56,name=''):
    """adds direction to dataframe, can be one of eight categories (N,S,W,E)"""
    #straight up (north)
    normal_vector = np.array([0,1])
    #normal_vector = np.tile(normal_vector,(df.shape[0],1))
    #normal_vector = normal_vector.reshape((2,-df.shape[0]))
    temp = pd.DataFrame()
    temp['latitude'] = df['latitude']-latitude
    temp['longitude'] = df['longitude']-longitude


    apartment_vector = temp[['latitude','longitude']].to_numpy()


    #print(np.shape(apartment_vector), np.shape(normal_vector))
    angles = []
    for vector in apartment_vector:
        if vector[0] < 0:
            temp_angle = -angle_between(vector,normal_vector)
        else:
            temp_angle = angle_between(vector,normal_vector)
        angles.append(temp_angle)

    angles = [element * 10 for element in angles]


    angles_series = pd.Series(np.array(angles))
    #angles_series.plot.hist()

    df['angle'+name] = angles_series
    max = df['angle'+name].max()
    min = df['angle'+name].min()
    bins = [min,min*7/8,min*5/8,min*3/8,min/8,max/8,max*3/8,max*5/8,max*7/8,max]
    rounded_bins = [element for element in bins]
    #print(rounded_bins)
    direction = pd.cut(df['angle'+name], bins= rounded_bins,labels=['S','SW','W','NW','N','NE','E','SE','S'],ordered=False)
    df['direction'+name] = direction
    return df

def unit_vector(vector):
    """returns the unit vector if the vector"""
    return vector/np.linalg.norm(vector)

def angle_between(v1,v2):
    """returns angle between two vectors in radian"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u,v2_u),-1,1))

def fix_geo_data(data_test):
    """should just be called on test data"""
    #fix all radius issues
    data_test._set_value(23,'longitude',37.473761)
    data_test._set_value(23,'latitude',55.560891)
    data_test._set_value(90,'longitude',37.473761)
    data_test._set_value(90,'latitude',55.560891)

    data_test._set_value(2511,'longitude',37.478055)
    data_test._set_value(2511,'latitude',55.544046)
    data_test._set_value(5090,'longitude',37.478055)
    data_test._set_value(5090,'latitude',55.544046)
    data_test._set_value(6959,'longitude',37.478055)
    data_test._set_value(6959,'latitude',55.544046)
    data_test._set_value(8596,'longitude',37.478055)
    data_test._set_value(8596,'latitude',55.544046)

    data_test._set_value(4719,'longitude',37.385493)
    data_test._set_value(4719,'latitude',55.853117)

    data_test._set_value(9547,'longitude',37.384711)
    data_test._set_value(9547,'latitude',55.853511)

    data_test._set_value(2529,'longitude',37.464994)
    data_test._set_value(2529,'latitude',55.627666)

    data_test = add_radius(data_test)

    return data_test

def drop_n_largest(data_train):
    """drops 4 largest values, should only be called on training set"""
    #drop 4 largest from training data, maybe not samrt, but we will see
    data_train.drop([3217,21414,15840,13938])
    return data_train

def clean_data(all_data):
    """cleans the data with all the knowledge we have so far"""


    #fix ceiling issues
    all_data.loc[all_data['ceiling'] > 50,'ceiling']*=0.01
    all_data.loc[all_data['ceiling'] > 25, 'ceiling']*=0.1
    all_data.loc[all_data['ceiling'] < 0.5,'ceiling'] = float('NaN')
    """"
    #fix area_kitchen and area_living issues
    all_data['living'] = all_data.area_living/all_data.area_total
    all_data['kitchen'] = all_data.area_total/all_data.area_kitchen

    all_data.loc[all_data['living'] > 1,'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['living'] > 1,'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100

    all_data.loc[all_data.area_living/all_data.area_total > 1, 'area_living'] = float('NaN')

    all_data['living'] = all_data.area_living/all_data.area_total
    all_data['kitchen'] = all_data.area_kitchen/all_data.area_total

    all_data['sum_area'] = all_data.area_living + all_data.area_kitchen
    all_data.loc[all_data['sum_area'] == 100, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 100, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100

    #this fixed some few rows.
    #when printing we see some other very clear "precentage situations, fixing these
    all_data.loc[all_data['sum_area'] == 38.5, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 38.5, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 52.7, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 52.7, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 71.6, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 71.6, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 20), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 20), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 15), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 15), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 10), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 10), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 30), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 30), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 10), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 10), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 15), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 15), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 20), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 20), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 25), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 25), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 30), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 30), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 50), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 50), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 60) & (all_data['area_total'] > 120), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 60) & (all_data['area_total'] > 120), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] < 70) & (all_data['area_total'] > 120)&(all_data['rooms'] < 3), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] < 70) & (all_data['area_total'] > 120)&(all_data['rooms'] < 3), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    """
    return all_data

def add_high_up(df):
    """add exponetital function to determine how high up a building is"""
    high_up = df.floor/df.stories
    high_up_exp = np.exp(high_up) - 1
    euler = np.exp(1)

    df['high_up'] = high_up_exp
    df['high_up'].where(df['high_up'] > euler, euler)


    return df
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    """base models must be given in order: XGB_boost, lightGBM, CatBoost, RandomForest and gradientBoost"""
    def __init__(self, base_models, meta_model, n_folds=10):
        self.base_models = np.array(base_models)
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.base_models_ = []
        for i in self.base_models:
            self.base_models_.append([])
   
    # We again fit the data on clones of the original models
    def fit(self, X_xgb, X_light, light_feat,X_cat, cat_feat, y):
        
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X_xgb.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            print('kjørrrr')
            if i == 0:
                for train_index, holdout_index in kfold.split(X_xgb, y):
                    instance = clone(model)
                    print('xgb')
                    self.base_models_[i].append(instance)
                    print(train_index)
                    instance.fit(X_xgb.iloc[train_index], y.iloc[train_index])
                    y_pred = instance.predict(X_xgb.iloc[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred
            if i == 1:
                for train_index, holdout_index in kfold.split(X_light, y):
                    instance = clone(model)
                    print('light')
                    self.base_models_[i].append(instance)
                    instance.fit(X_light.iloc[train_index], y.iloc[train_index],categorical_feature=light_feat)
                    y_pred = instance.predict(X_light.iloc[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred
            if i == 2:
                for train_index, holdout_index in kfold.split(X_cat, y):
                    instance = clone(model)
                    print('cat')
                    self.base_models_[i].append(instance)
                    instance.fit(X_cat.iloc[train_index], y.iloc[train_index],cat_features=cat_feature)
                    y_pred = instance.predict(X_cat.iloc[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred
            if i == 3:
                for train_index, holdout_index in kfold.split(X_xgb, y):
                    instance = clone(model)
                    print('luring1')
                    self.base_models_[i].append(instance)
                    instance.fit(X_xgb.iloc[train_index], y.iloc[train_index])
                    y_pred = instance.predict(X_xgb.iloc[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred
            if i == 4:
                for train_index, holdout_index in kfold.split(X_xgb, y):
                    instance = clone(model)
                    print('luring2')
                    self.base_models_[i].append(instance)
                    instance.fit(X_xgb.iloc[train_index], y.iloc[train_index])
                    y_pred = instance.predict(X_xgb.iloc[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X_xgb_test, X_light_test,X_cat_test):
        for i, base_models in enumerate(self.base_models_):
            print(i)
            if i == 0:
                pred_model = []
                for model in base_models:
                    pred_model.append(model.predict(X_xgb_test))
                av_model_0 = np.column_stack(pred_model).mean(axis=1)
            if i == 1:
                pred_model = []
                for model in base_models:
                    pred_model.append(model.predict(X_light_test))
                av_model_1 = np.column_stack(pred_model).mean(axis=1)
            if i == 2:
                pred_model = []
                for model in base_models:
                    pred_model.append(model.predict(X_cat_test))
                av_model_2 = np.column_stack(pred_model).mean(axis=1)
            if i == 3:
                pred_model = []
                for model in base_models:
                    pred_model.append(model.predict(X_xgb_test))
                av_model_3 = np.column_stack(pred_model).mean(axis=1)
            if i == 4:
                pred_model = []
                for model in base_models:
                    pred_model.append(model.predict(X_xgb_test))
                av_model_4 = np.column_stack(pred_model).mean(axis=1)
        meta_features = np.column_stack([av_model_0,av_model_1,av_model_2,av_model_3,av_model_4])
        return self.meta_model_.predict(meta_features)

def log_radius(df):
    df['radius'] = np.log(df['radius'])
    return df

def cluster_geo_data(df,df_test):
    from sklearn.cluster import KMeans
    k_means = KMeans(n_clusters = 100, max_iter = 1000, init='k-means++')

    lat_long_pairs = df[['latitude','longitude']]
    lat_long_pairs_test = df_test[['latitude','longitude']]
    target_data = np.log(df.price)/np.log(15)

    k_means.fit(lat_long_pairs,sample_weight = target_data)
    df['cluster_number'] = k_means.predict(lat_long_pairs)
    df_test['cluster_number'] = k_means.predict(lat_long_pairs_test)

    return df, df_test

I want to use import_data function from another directory, and make the data ready for the model. LightGBM accepts categorical values, but they have to be encoded as no-negative integers.

In [3]:
#We dont want our model to care about the id of the house or the seller
data, data_test = import_data()
data_light, data_test_light = import_data()

Y = data.price
test_id = data_test.id

radius = True
cleaning = True

if(cleaning):
    data_test = fix_geo_data(data_test)
    data = clean_data(data)
    data_test = clean_data(data_test)

if(radius):
    data = add_radius(data)
    data_test = add_radius(data_test)
    #coordinates = ['latitude', 'longitude']
    #data = data.drop(columns = coordinates)
    #data_test = data_test.drop(columns = coordinates)
    #data_light = add_radius(data_light)
    #data_test_light = add_radius(data_test_light)

fix_missing = False
if (fix_missing):
    data = fix_missing_value(data)
    data_test = fix_missing_value(data_test)
    #data_light = fix_missing_value(data_light)
    #data_test_light = fix_missing_value(data_test_light)


    #data_test_light = fix_geo_data(data_test_light)
    #data_light = clean_data(data_light)
    #data_test_light = clean_data(data_test_light)

school = True
park = True
metro = True
attraction = True
train = True
more_radians = True
if (more_radians):
    data = add_parks_and_schools(data)
    data_test = add_parks_and_schools(data_test)

if (school):
    data = closest_school(data)
    data_test = closest_school(data_test)

if (park):
    data = closest_park(data)
    data_test = closest_park(data_test)

if (train):
    data = closest_train(data)
    data_test = closest_train(data_test)

if (metro):
    data = closest_metro(data)
    data_test = closest_metro(data_test)

if (attraction):
    data = closest_attraction(data)
    data_test = closest_attraction(data_test)
data_from = data
data_test_from = data_test
print(data_from.info())

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [4]:
data_test_light = data_test_from
data_light = data_from
data_test = data_test_from
data = data_from

print(data_light['seller'].dtype)

categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'building_id','new','district','street',
                    'address', 'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','layout','direction','direction0','direction1','direction2','direction3','direction4','direction5','direction6','direction7','direction8','direction9','direction10','direction11','direction12','direction13','direction14','direction15']

data_columns = list(data.columns)
numerical_data = list(set(data_columns)-set(categorical_data))
print(numerical_data)
for column in numerical_data:
    print(column)
    mean = data[column].mean()
    data[column] = data[column].replace(np.NaN, mean)
    if column != 'price':
        mean_test = data_test[column].mean()
        data_test[column] = data_test[column].replace(np.NAN,mean)

#Features
radius = True
district_mean_price = True
ohe = True
direction = True
high_up = True
#add high up
log_r = False
cluster = True

print(data_light['seller'].dtype)

if(high_up):
    data = add_high_up(data)
    data_test = add_high_up(data_test)
    data_light = add_high_up(data_light)
    data_test_light = add_high_up(data_test_light)
print(data_light['seller'].dtype)
if(cluster):
    data['price']= Y
    data_light['price']= Y
    data,data_test = cluster_geo_data(data,data_test)
    data_light,data_test_light = cluster_geo_data(data_light,data_test_light)

print(data_light['seller'].dtype)
if(district_mean_price):
    data = group_by_feature_and_price(data,'cluster_number')
    data_test = group_by_feature_and_price(data_test,'cluster_number',data_train=data)
    data_light = group_by_feature_and_price(data_light,'cluster_number')
    data_test_light = group_by_feature_and_price(data_test_light,'cluster_number', data_train=data_light)

print(data_light['seller'].dtype)

if(direction):
    data = add_direction(data)
    data_test = add_direction(data_test)


    data_light = add_direction(data_light)
    data_test_light = add_direction(data_test_light)


if(log_r):
    data = log_radius(data)
    data_test = log_radius(data_test)
    data_light = log_radius(data_light)
    data_test_light = log_radius(data_test_light)
 
data_from = data.copy()
data_test_from = data_test.copy()
data_light_from = data_light.copy()
data_light_test_from = data_test_light.copy()
print(data_light['seller'].dtype)
print('done here')

float64
['radius1', 'angle13', 'radius2', 'distance_park', 'angle10', 'floor', 'loggias', 'stories', 'radius', 'closest_train_station', 'closest_metro_station', 'radius11', 'bathrooms_private', 'distance_train_station', 'angle15', 'area_living', 'distance_atraction', 'radius12', 'radius8', 'radius5', 'angle1', 'distance_school', 'radius6', 'angle14', 'longitude', 'angle5', 'angle8', 'bathrooms_shared', 'angle11', 'radius10', 'angle7', 'radius15', 'area_kitchen', 'area_total', 'price', 'closest_atraction', 'angle4', 'ceiling', 'angle12', 'radius3', 'id', 'angle6', 'radius4', 'radius7', 'rooms', 'radius13', 'angle0', 'constructed', 'balconies', 'angle9', 'latitude', 'angle3', 'closest_park', 'radius9', 'phones', 'angle2', 'closest_school', 'radius14', 'distance_metro_station', 'radius0']
radius1
angle13
radius2
distance_park
angle10
floor
loggias
stories
radius
closest_train_station
closest_metro_station
radius11
bathrooms_private
distance_train_station
angle15
area_living
distance_atrac

In [5]:
data = data_from.copy() 
data_test = data_test_from.copy()
data_light = data_light_from.copy()
data_test_light = data_light_test_from.copy()
print('xgb_boost')
data.info()
data_test.info()
print('light')
data_light.info()
data_test_light.info()
print(data_test.columns)

xgb_boost
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23285 entries, 0 to 23284
Data columns (total 98 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   id                      23285 non-null  int64   
 1   seller                  14455 non-null  float64 
 2   price                   23285 non-null  float64 
 3   area_total              23285 non-null  float64 
 4   area_kitchen            23285 non-null  float64 
 5   area_living             23285 non-null  float64 
 6   floor                   23285 non-null  float64 
 7   rooms                   23285 non-null  float64 
 8   layout                  5643 non-null   float64 
 9   ceiling                 23285 non-null  float64 
 10  bathrooms_shared        23285 non-null  float64 
 11  bathrooms_private       23285 non-null  float64 
 12  windows_court           15213 non-null  float64 
 13  windows_street          15213 non-null  float64 
 14  balconies   

In [50]:
#features which can be added
radius = ['radius','radius0','radius1','radius2','radius3','radius4','radius5','radius6','radius7','radius8','radius9','radius10','radius11','radius12','radius13','radius14','radius15']
angles = ['angle0','angle1','angle2','angle3','angle4','angle5','angle6','angle7','angle8','angle9','angle10','angle11','angle12','angle13','angle14','angle15']
#drop n largest
drop = False
if (drop):
    data = drop_n_largest(data)
    data_cat = drop_n_largest(data_cat)
    data_light = drop_n_largest(data_light)

#Here we choose which features to focus on
if False:
    """lets first try all features for all machines"""
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,4,10,11,12,13,14,15,17,19,20,21,22,23,24,42,44,46,49,52,55)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,4,10,11,12,13,14,15,17,19,20,21,22,23,24,42,44,46,49,52,55]

    
if False:
    """lets try to only focus on goe data"""
    focus_xgb = ['area_total','radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
        'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['area_total','radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
        'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (18,20,22,25,27,30)
    
    focus_cat = ['area_total','radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
        'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [18,20,22,25,27,30]
    
if False:
    """lets try to look at some more features as categories"""
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['rooms','bathrooms_shared', 'bathrooms_private','seller','layout', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction','constructed']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,3,4,6,7,10,11,12,13,14,15,16,17,19,20,21,22,23,24,42,44,46,49,52,55)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,3,4,6,7,10,11,12,13,14,15,16,17,19,20,21,22,23,24,42,44,46,49,52,55]

if False:
    """lets try to look at eeeeeven more features as categories, stories and floors"""
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['rooms','floor','stories','bathrooms_shared', 'bathrooms_private','seller','layout', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction','constructed']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,2,3,4,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,42,44,46,49,52,55)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,2,3,4,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,42,44,46,49,52,55]    

if False:
    """keep best one for each so far"""
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,3,4,6,7,10,11,12,13,14,15,16,17,19,20,21,22,23,24,42,44,46,49,52,55)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'garbage_chute', 'heating', 'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,4,10,11,12,13,14,15,17,19,20,21,22,23,24,42,44,46,49,52,55]
    
if False:
    """lets start removing columns we think make the result worse"""
    #remove garbage_chute and heating
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,3,4,6,7,10,11,12,13,14,15,16,17,19,20,21,22,40,42,44,47,50,53)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'layout', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,4,10,11,12,13,14,15,17,19,20,21,22,40,42,44,47,50,53]

if False:
    """lets continue removing bad values"""
    #remove layout
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['seller', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,3,5,6,9,10,11,12,13,14,15,16,18,19,20,21,39,41,43,46,49,52)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,9,10,11,12,13,14,16,18,19,20,21,39,41,43,46,49,52]
    

if False:
    """lets continue removing bad values"""
    #remove some geo data, start with distance_metro_station
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['seller', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,3,5,6,9,10,11,12,13,14,15,16,18,19,20,21,39,41,43,45,48,51)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
        'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,9,10,11,12,13,14,16,18,19,20,21,39,41,43,45,48,51]
    
if False:
    """lets continue removing bad values"""
    #remove mean_distict_price
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'angle', 'direction', 'distance_metro_station']
    
    xgb_categorical = ['seller', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'angle', 'direction', 'distance_metro_station']
    
    #heating 24
    light_features = (0,3,5,6,9,10,11,12,13,14,15,16,18,19,20,21,39,41,43,45,48,50)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
        'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'angle', 'direction', 'distance_metro_station']
    cat_feature = [0,9,10,11,12,13,14,16,18,19,20,21,39,41,43,45,48,50]

if False:
    """optimal solution for now, want to perfect weighting"""

    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    xgb_categorical = ['seller', 'windows_court', 'windows_street', 'condition', 'new','district',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park','closest_train_station','direction','closest_atraction']
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    
    #heating 24
    light_features = (0,3,5,6,9,10,11,12,13,14,15,16,18,19,20,21,39,41,43,46,49,52)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones', 'new', 'district',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'distance_school', 'closest_park',
       'distance_park', 'closest_train_station', 'distance_train_station',
       'distance_metro_station', 'closest_atraction', 'distance_atraction',
       'high_up', 'cluster_number', 'district_price', 'angle', 'direction']
    cat_feature = [0,9,10,11,12,13,14,16,18,19,20,21,39,41,43,46,49,52]

if False:
    """maybe there is to many things saying the same, lets try to drop some info"""
    #dropping district,direction, attraction things, train things, distance park and school
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'closest_park',
       'distance_metro_station',
       'high_up', 'cluster_number', 'cluster_number_price', 'angle']
    xgb_categorical = ['seller', 'windows_court', 'windows_street', 'condition', 'new',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park',]
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'closest_park',
       'distance_metro_station',
       'high_up', 'cluster_number', 'cluster_number_price', 'angle']
    
    #heating 24
    light_features = (0,3,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,38,39,42)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'closest_park',
       'distance_metro_station',
       'high_up', 'cluster_number', 'cluster_number_price', 'angle']
    cat_feature = [0,7,8,9,10,11,12,13,14,15,17,18,19,20,38,39,42]

if True:
    """maybe there is to many things saying the same, lets try to drop some info"""
    #dropping district,direction, attraction things, train things, distance park and schoolp, add phones to cat features
    focus_xgb = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'closest_park',
       'distance_metro_station',
        'cluster_number', 'cluster_number_price', 'angle','high_up','angle0','angle1','angle2','angle3','angle4','angle5','angle6','angle7','angle8','angle9','angle10','angle11','angle12','angle13','angle14','angle15']
    xgb_categorical = ['seller', 'windows_court', 'windows_street', 'condition', 'new', 'phones',
                      'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','cluster_number','closest_school',
                      'closest_park',]
    
    focus_light = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'closest_park',
       'distance_metro_station',
       'cluster_number', 'cluster_number_price', 'angle','high_up','angle0','angle1','angle2','angle3','angle4','angle5','angle6','angle7','angle8','angle9','angle10','angle11','angle12','angle13','angle14','angle15']
    
    #heating 24
    light_features = (0,3,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,38,39,41)
    
    focus_cat = ['seller', 'area_total','floor',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'condition',
       'phones','new',
       'constructed', 'material', 'stories',
       'elevator_without', 'elevator_passenger', 'elevator_service', 'parking',
       'radius', 'radius0', 'radius1', 'radius2',
       'radius3', 'radius4', 'radius5', 'radius6', 'radius7', 'radius8',
       'radius9', 'radius10', 'radius11', 'radius12', 'radius13', 'radius14',
       'radius15', 'closest_school', 'closest_park',
       'distance_metro_station',
       'cluster_number', 'cluster_number_price', 'angle','high_up','angle0','angle1','angle2','angle3','angle4','angle5','angle6','angle7','angle8','angle9','angle10','angle11','angle12','angle13','angle14','angle15']
    cat_feature = [0,7,8,9,10,11,12,13,14,15,17,18,19,20,38,39,41]

#encode_categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition','new','district',
 #     'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number']
#categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'building_id','new','district','street',
#                    'address', 'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating','cluster_number','layout']



def normalize_numerical(df,df_test, numerical_columns):
    temp = df[numerical_columns]
    temp_test = df_test[numerical_columns]
    for column in numerical_columns:

        min_df = min([temp[column].min(),temp_test[column].min()])
        max_df = max([temp[column].max(),temp_test[column].max()])

        temp[column] = (temp[column]-min_df)/(max_df-min_df)
        temp_test[column] = (temp_test[column]-min_df)/(max_df-min_df)

    df[numerical_columns] = temp
    df_test[numerical_columns] = temp_test
    

    return df,df_test
 
#radius = ['radius0','radius1','radius2','radius3','radius4','radius5','radius6','radius7','radius8','radius9','radius10','radius11','radius12','radius13','radius14','radius15']
    
#focus_light = ['cluster_number','seller','material','district','condition','parking','new','direction','constructed','elevator_without','garbage_chute','loggias','layout','balconies','floor','high_up','rooms','area_total','radius','ceiling','angle','distance_metro_station']
#light_features = (0,1,2,3,4,5,6,7,8,9,10,11,12,13)
#focus_cat = ['elevator_passenger', 'elevator_service','windows_court', 'windows_street','cluster_number','seller','material','district','condition','parking','new','direction','constructed','elevator_without','garbage_chute','loggias','layout','balconies','floor','high_up','rooms','area_total','radius','ceiling','angle','distance_metro_station']
#categorical_cat = ['elevator_passenger', 'elevator_service','windows_court', 'windows_street','cluster_number','seller','material','district','condition','parking','new','direction','constructed','elevator_without','garbage_chute','loggias','layout','balconies']
#cat_feature = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
#focus_light +=radius
#focus_cat +=radius
print('done')

done


In [51]:
data_focus = data[focus_xgb]
data_test_focus = data_test[focus_xgb]


data_train_focus_light = data_light[focus_light]
data_test_focus_light = data_test_light[focus_light]

print(data_train_focus_light['seller'].dtype, data_light['seller'].dtype)

#cat data should not be one hot encoded, and it doesnt accept nan values
data_cat, data_cat_test = data, data_test
print(data_train_focus_light['seller'].dtype, data_light['seller'].dtype)
for column in data_cat.columns:
    data_cat[column] = data_cat[column].replace(np.NaN, 'None')
    if column !='price':
        data_cat_test[column] = data_cat_test[column].replace(np.NaN, 'None')

print(data_train_focus_light['seller'].dtype, data_light['seller'].dtype)

ohe = True
#One hot encode data
if(ohe):
    data_focus = one_hot_encode_multiple(data_focus,xgb_categorical)
    data_test_focus = one_hot_encode_multiple(data_test_focus,xgb_categorical)
    
    #Drop cat_data
    data_focus = data_focus.drop(columns = xgb_categorical)
    data_test_focus = data_test_focus.drop(columns = xgb_categorical)
   

#make sure all columns are equal in data_test and data_train for xgboost
not_encoded_test = list(set(data_focus.columns)-set(data_test_focus.columns))
not_encoded_train = list(set(data_test_focus.columns)-set(data_focus.columns)) 
print(data_train_focus_light['seller'].dtype, data_light['seller'].dtype)
for column in not_encoded_test:
    data_test_focus[column] = np.zeros(data_test_focus.shape[0])
for column in not_encoded_train:
    data_focus[column] = np.zeros(data_focus.shape[0])

#change cat_data to str for catboost
categorical_cat = []
for index in cat_feature:
    categorical_cat.append(focus_cat[index])

for column in categorical_cat:
    data_cat[column]= data_cat[column].astype(str)
    data_cat_test[column]= data_cat_test[column].astype(str)

#chose which columns to foucs on light gbm,catboost
data_train_focus_cat = data_cat[focus_cat]
data_test_focus_cat = data_cat_test[focus_cat]


    
normalize = True
#normalize numerical data
numerical_cat = []
categorical_cat = []
numerical_xgb = []
categorical_xgb = xgb_categorical
numerical_light = []
categorical_light = []
for index in cat_feature:
    categorical_cat.append(focus_cat[index])
for index in list(light_features):
    categorical_light.append(focus_light[index])


#didnt work to normalize all, but want to try to normalize all radiuses
numerical_cat = list(set(focus_cat)-set(categorical_cat))
numerical_light = list(set(focus_light)-set(categorical_light))
numerical_xgb = list(set(focus_xgb)-set(categorical_xgb))


data_focus[['constructed','loggias','balconies']] = data_focus[['constructed','loggias','balconies']].astype(float)
data_test_focus[['constructed','loggias','balconies']] = data_test_focus[['constructed','loggias','balconies']].astype(float)
print(numerical_cat,numerical_light,numerical_xgb)
if (normalize):
    data_focus,data_test_focus = normalize_numerical(data_focus,data_test_focus,radius)
    data_train_focus_cat, data_test_focus_cat = normalize_numerical(data_train_focus_cat, data_test_focus_cat,radius)
    data_train_focus_light, data_test_focus_light = normalize_numerical(data_train_focus_light, data_test_focus_light,radius)
    
X_train, X_test, y_train, y_test = train_test_split(data_focus, Y, test_size=0.2, random_state=42)
X_train_light, X_test_light, y_train_light, y_test_light = train_test_split(data_train_focus_light, Y, test_size=0.2, random_state=42)
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data_train_focus_cat, Y, test_size=0.2, random_state=42)
print('done')


float64 float64
float64 float64
float64 float64
float64 float64
['radius1', 'angle13', 'radius2', 'floor', 'angle10', 'stories', 'radius', 'radius11', 'bathrooms_private', 'angle15', 'radius12', 'radius8', 'radius5', 'angle1', 'radius6', 'cluster_number_price', 'angle14', 'angle5', 'bathrooms_shared', 'radius15', 'angle8', 'radius10', 'angle7', 'angle11', 'area_total', 'angle4', 'ceiling', 'angle', 'angle12', 'radius3', 'angle6', 'radius4', 'radius7', 'rooms', 'radius13', 'angle0', 'high_up', 'angle9', 'angle3', 'radius9', 'angle2', 'radius14', 'distance_metro_station', 'radius0'] ['radius1', 'angle13', 'radius2', 'floor', 'angle10', 'stories', 'radius', 'radius11', 'angle15', 'radius12', 'radius8', 'radius5', 'angle1', 'radius6', 'cluster_number_price', 'angle14', 'angle5', 'radius15', 'angle8', 'angle11', 'radius10', 'angle7', 'area_total', 'angle4', 'ceiling', 'angle', 'angle12', 'radius3', 'angle6', 'radius4', 'radius7', 'radius13', 'angle0', 'high_up', 'angle9', 'angle3', 'radius9

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:624: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:625: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [58]:
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

print('go')
#data.info()
#print(len(Y))
#print(data.corr())
kaggle = True
if(kaggle):
    Y_use = np.log(Y)/np.log(15)
 
    y_train_use = Y_use
    
    
    #define different base models
    rf_model = RandomForestRegressor(random_state = 42, n_estimators = 300,n_jobs = -1)
    
    XGB_model = xgb.XGBRegressor(n_estimators = 3000,eta=0.15, random_state=42, max_depth =4,n_jobs=-1)
    
    lightGBM_model_2 = lgb.LGBMRegressor(
    categorical_feature=light_features,
    num_leaves=7**2,
    max_depth=7,
    random_state=42,
    metric='rmsle',
    num_iterations=2500,
    learning_rate=0.07,
    n_jobs = -1)
    
    cat = CatBoostRegressor(loss_function= 'RMSE',verbose=False,random_state = 42, depth=9, learning_rate=0.09, iterations = 2500)
    
    gradientBoost = GradientBoostingRegressor(random_state= 42, max_depth = 9, min_samples_leaf=4, learning_rate=0.06, n_estimators=400)
    
    base_models = (XGB_model, lightGBM_model_2, cat, rf_model, gradientBoost)
    meta_model = LinearRegression()
    
    stack = StackingAveragedModels(base_models, meta_model)
    
    fit_stack = stack.fit(data_focus,data_train_focus_light, light_features, data_train_focus_cat, cat_feature, y_train_use)
    print('ferdig fit')
    prediction_kaggle = fit_stack.predict(data_test_focus,data_test_focus_light,data_test_focus_cat)
    print('ferdig')
    
else:
    y_train_use = np.log(y_train)/np.log(15)
    
    
    #define different base models
    rf_model = RandomForestRegressor(random_state = 42,n_jobs = -1)
    
    XGB_model = xgb.XGBRegressor(random_state=42,n_jobs=-1)
    
    lightGBM_model_2 = lgb.LGBMRegressor(
    categorical_feature=light_features,
    random_state=42,
    n_jobs = -1)
    
    cat = CatBoostRegressor(verbose=False,random_state = 42)
    
    gradientBoost = GradientBoostingRegressor(random_state= 42)
    
    base_models = (XGB_model, lightGBM_model_2, cat, rf_model, gradientBoost)
    meta_model = LinearRegression()
    
    stack = StackingAveragedModels(base_models, meta_model)
    
    fit_stack = stack.fit(X_train,X_train_light, light_features, X_train_cat, cat_feature, y_train_use)
    print('ferdig fit')
    prediction = fit_stack.predict(X_test,X_test_light,X_test_cat)
    print('ferdig')

go
kjørrrr
xgb
[    0     1     2 ... 23282 23283 23284]
xgb
[    0     1     2 ... 23281 23282 23283]
xgb
[    0     2     3 ... 23282 23283 23284]
xgb
[    1     2     3 ... 23282 23283 23284]
xgb
[    0     1     2 ... 23281 23282 23284]
xgb
[    0     1     2 ... 23282 23283 23284]
xgb
[    0     1     2 ... 23282 23283 23284]
xgb
[    0     1     4 ... 23281 23283 23284]
xgb
[    0     1     2 ... 23282 23283 23284]
xgb
[    0     1     2 ... 23282 23283 23284]
kjørrrr
light


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1224: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_feature in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


light
light
light
light
light
light
light
light
light
kjørrrr
cat
cat
cat
cat
cat
cat
cat
luring1
luring1
luring1
luring1
luring1
luring1
luring1
luring1
kjørrrr
luring2
luring2
luring2
luring2
luring2
luring2
luring2
luring2
luring2
luring2
ferdig fit
0
1
2
3
4
ferdig


In [59]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
def rmsle_cv(model, train,y_train):
    kf = KFold(5, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)
#lgb.plot_importance(lightGBM_model)
#print(data.test)

if(kaggle):
    prediction_kaggle_final = 15**prediction_kaggle
    submission = pd.DataFrame()
    submission['id'] = test_id
    submission['price_prediction'] = prediction_kaggle_final
    submission.to_csv('submission_XGB_EDA.csv', index= False)
    for i, list in enumerate(predictions):
        submission = pd.DataFrame()
        submission['id'] = test_id
        submission['price_prediction'] = 15**list
        name = 'submission'+str(i)+'.csv'
        submission.to_csv(name, index= False)
        
else:
    final_prediction = 15**prediction
    rmsle = root_mean_squared_log_error(y_test,final_prediction)
    print("first run", rmsle)

NameError: name 'predictions' is not defined

optimalisering av vekter:
    0.1187787092957582 (0.15 0.1 0.4 0.15 0.2) 
    

Runs

   - 0.121 med radians og metro

   - stacking med fire varianter og clustering 0.125, gir xx på kaggle
   - Alle kolonner: tot: 0.12059515091457082 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.129
                    xgb_1: 0.135
                    xgb_2: 0.123
                    light_1: 0.131
                    light_2: 0.129 
    - Alle kolonner (med ekstra kolonner som kategori (rooms, constructed,'bathrooms_shared', 'bathrooms_private') 
                    tot:0.12220117287391123 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.131
                    xgb_1: 0.136
                    xgb_2: 0.125
                    light_1: 0.129
                    light_2: 0.129 
    - Alle kolonner (med ekstra kolonner som kategori (rooms, constructed,'bathrooms_shared', 'bathrooms_private','floors,'stories) )
                    tot:0.1218372816191243 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.132
                    xgb_1: 0.135
                    xgb_2: 0.1259
                    light_1: 0.131
                    light_2: 0.132
 
     - Alle kolonner (valgt kategorier for å optimalisere for hver modell)
                    tot:0.11971322837704079 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.129
                    xgb_1: 0.131
                    xgb_2: 0.125
                    light_1: 0.129
                    light_2: 0.129
    
    - Fjernet heating og garbage_chute med optimale
                    tot:0.11933026516940594 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.129
                    xgb_1: 0.130
                    xgb_2: 0.124
                    light_1: 0.129
                    light_2: 0.129
 
    - Fjernet heating og garbage_chute og layout og mean_price med optimale
                    tot:0.11964331168413234 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.130
                    xgb_1: 0.132
                    xgb_2: 0.125
                    light_1: 0.1282
                    light_2: 0.128
 
    - Fjernet heating og garbage_chute og layout med optimale
                    tot:0.11923639644076009 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.129
                    xgb_1: 0.131
                    xgb_2: 0.1238
                    light_1: 0.129
                    light_2: 0.1274

    - Fjernet heating og garbage_chute og layout og closest_metro med optimale
                    tot:0.1210324459397781 (vekting 0.15,0.35,0.25,0.15,0.10)
                    cat: 0.131
                    xgb_1: 0.134
                    xgb_2: 0.125
                    light_1: 0.129
                    light_2: 0.128
              
    - Kun geodata: tot:0.14513708211825474 (vekting 0.15,0.35,0.25,0.15,0.10)
                   cat: 0.151
                   xgb_1: 0.156
                   xgb_2: 0.152
                   light_1: 0.145
                   light_2: 0.152
     

på kaggle     - Fjernet heating og garbage_chute og layout med optimale (0.15 0.1 0.4 0.15 0.2) 
                   tot: 0.160
                   cat: 0.168
                   xgb_1: 0.18
                   xgb_2: 0.167
                   light_1: 0.171
                   light_2: 0.171
                   
på kaggle     - Fjernet mye data, feks direction, district, og mye geodata, prøvde å ta vare på de vi tror på (0.15 0.1 0.4 0.15 0.2) 
                   tot: 0.15509
                   cat: 0.163
                   xgb_1: 0.175
                   xgb_2: 0.162
                   light_1: 0.171
                   light_2: 0.171
                 forslag til nye vekter (0.25,0.1,0.4,0.1,0.,0.15) tot: 0.15406
                 la oss la de beste dominere enda mer(0.3,0.05,0.45,0.1,0.1) tot: 0.15417
                 
                 endret vekter: økte num_operators og endret noen småting - 0.15402 (ny vekting 0.3 0.05 0.4 0.15 0.1) tot: 0.15361
                   tot: 0.15402
                   cat: 0.159
                   xgb_1: 0.182
                   xgb_2: 0.162
                   light_1: 0.169
                   light_2: 0.171
                   random_forest = 0.17650
                   
                   


In [11]:
lokale output: samme som på beste på kaggle, max_depth = 5 på xgboost
first run 0.121405546173477
0.13303302594229785
0.15492257195124284
0.12407193048647067
0.13070451145172166
0.12871240618760926

max_depth = 4 på xgb_boost
first run 0.12005143208312112
                   cat: 0.12667498872149563
                   random_forest: 0.15524921799064284
                   xgb_2: 0.12364216811303258
                   light_1: 0.12985798951428007
                   light_2: 0.
                    
                    
                    
xgb meta : first run 0.13121136456942828
logistig regression meta : first run 0.12121445634333107
uten_noen_hyperparametre logistic: first run 0.13142465040526247
med lightGBM som meta: first run 0.13383483000683086
med k-fold lik 10 : first run 0.13069207544113803

oof på kaggle med logistig regression: 0.15365

lokal med alle angles: first run 0.1308
lokal med angle på alle radiuser og kfold=10: first run 0.12940370664902648

lokal med normalization av alle numerical: dårlig
lokal med normalizatio av alle radius og angles, kfold=10: first run 0.12939596781880774

SyntaxError: invalid syntax (415406153.py, line 1)